In [ ]:
import numpy as np
import pandas as pd
import requests
import json
import os
import ta

pd.set_option('display.max_columns', 100)


In [ ]:
EXCHANGE_ID = "2eccdb22-ec97-4799-9755-77c976991af8"

BTCUSDT_DAY_ID = "2b043a89-de72-4920-a375-fb5991ac405e"
BTCUSDT_HOUR_ID = "b6aebe76-a1f5-45ec-92a9-e059601473bc"
BTCUSDT_MINUTE_ID = "85d2cfef-ec7d-47cb-8969-439e9c58f352"

ETHUSDT_DAY_ID = "a6f19254-e396-4359-9fd6-b114216ffa06"
ETHUSDT_HOUR_ID = "2c94611b-e12e-4f27-beef-d4e9212f01e7"
ETHUSDT_MINUTE_ID = "4b4488bf-2c76-4e5c-9c71-e6e8f3cccae0"

BNBUSDT_DAY_ID = "e6bd7a29-c2e5-458e-8302-7afc0433f0c6"
BNBUSDT_HOUR_ID = "e4186eef-f39b-4746-9605-5e957c330ca3"
BNBUSDT_MINUTE_ID = "ec51c55e-4e60-4386-9498-b53d06189a30"


ASSET_ID = BTCUSDT_DAY_ID

base_url = "http://172.24.100.128:5000/"
# base_url = "http://localhost:5000/"


PROXIES = {
    
   'http': 'http://discproxy.virtual.uniandes.edu.co:443',
}


In [17]:
# Method to make API request to retrieve prices for a given asset ID between two specified Unix timestamps
def api_request_get_prices_between_unix_time(base_url, asset_id, unix_time_start, unix_time_end):

    # try:
    # Make GET request to API endpoint with query parameters for start and end Unix timestamps
    response = requests.get(base_url + "assets/" + asset_id + "/indicators_unix_between/",
                            params={'unix_time_start': unix_time_start, 'unix_time_end': unix_time_end},proxies=PROXIES)

    # Check if the request was successful
    if response.status_code == requests.codes.ok:
        # print("get prices between status code: "+str(response.status_code))
        # Convert JSON response to Python dictionary
        json_data = json.loads(response.content)
        # Normalize the dictionary and convert it to a Pandas DataFrame
        df = pd.json_normalize(json_data)

        # drop columns that are not needed
        df.drop(columns=['updated_at', 'created_at', 'date_time_gmt_5','ignore','asset_id'], inplace=True)

        # Return the DataFrame containing the retrieved price data
        return df

    else:
        # print("Failed api_request_get_prices_between_unix_time")
        print("get prices between status code: "+str(response.status_code))
        json_data = json.loads(response.content)
        print(json_data)
        return json_data
    


def api_request_get_asset_from_asset_id(base_url, exchange_id, asset_id):
    # try:
    # Make GET request to API endpoint

    response = requests.get(base_url + "exchanges/" +
                            exchange_id + "/asset/"+asset_id,proxies=PROXIES)

    # Check if the request was successful
    if response.status_code == requests.codes.ok:
        print("get asset status code: "+str(response.status_code))
        # Convert JSON response to Python dictionary
        json_data = json.loads(response.content)
        # Normalize the dictionary and convert it to a Pandas DataFrame
        df = pd.json_normalize(json_data)
        # Return the Unix timestamp of the last recorded price for the specified asset ID

        print("symbol: " + df["symbol"].iloc[0])
        print("interval: " + df["interval"].iloc[0])
        return df
    else:
        print(response.content)
        json_data = json.loads(response.content)
        df = pd.json_normalize(json_data)
        return df



# Method to make API request to retrieve prices for a given asset ID between two specified Unix timestamps
def api_request_get_all_indicators_from_price(base_url, price_id,):

    # try:
    # Make GET request to API endpoint with query parameters for start and end Unix timestamps
    response = requests.get(base_url + "prices/" + price_id + "/indicators/",proxies=PROXIES)

    # Check if the request was successful
    if response.status_code == requests.codes.ok:
        # Convert JSON response to Python dictionary
        json_data = json.loads(response.content)
        # Normalize the dictionary and convert it to a Pandas DataFrame
        df = pd.json_normalize(json_data)

        if df.empty:
            return df
        else:
        
            df.drop(columns=['id', 'created_at', 'updated_at','date_time_utc','unix_time','date_time_gmt_5'], inplace=True)

            df.set_index('price_id', inplace=True)

            df = df.replace(0, np.nan)

            return df

    else:
        print("get indicators status code: "+str(response.status_code))
        json_data = json.loads(response.content)
        print(json_data)
        return json_data
    

def get_indicators_and_merge(base_url,exchange_id, asset_id, unix_time_start, unix_time_end):


    # Get the asset from the API
    asset = api_request_get_asset_from_asset_id(base_url,exchange_id,asset_id)

    interval = asset["interval"].iloc[0]
    
    if interval == "minute":
        first_indicator_time = unix_time_start - 6000

    if interval == "hour":
        first_indicator_time = unix_time_start - 360000

    if interval == "day":
        first_indicator_time = unix_time_start - 8640000


    df = api_request_get_prices_between_unix_time(base_url, asset_id, first_indicator_time, unix_time_end)
    
    df_prices_wit_indicators_all = add_indicators(df)

    df_prices_wit_indicators = df_prices_wit_indicators_all.loc[df_prices_wit_indicators_all['unix_time'] >= unix_time_start]

    df_assets = pd.DataFrame()

    for i in df['id']:

        df2 = api_request_get_all_indicators_from_price(base_url, i)

        if not df2.empty:
            df_assets = pd.concat([df_assets, df2])
        
    if not df_assets.empty:
        df_merge = pd.merge(df_prices_wit_indicators, df_assets, left_on='id', right_on='price_id', how='left')
    else:
        df_merge = df_prices_wit_indicators

    return df_merge


# method to add the thecnic indicators to the dataframe
def add_indicators(df):

    # momentum
    df['ao'] = ta.momentum.awesome_oscillator(
        high=df['high_price'], low=df['low_price'])
    df['kama'] = ta.momentum.kama(close=df['close_price'])
    df['ppo'] = ta.momentum.ppo(close=df['close_price'])
    df['pvo'] = ta.momentum.pvo(volume=df['volume'])
    df['roc'] = ta.momentum.roc(close=df['close_price'])
    df['rsi'] = ta.momentum.rsi(close=df['close_price'])
    df['stochrsi'] = ta.momentum.stochrsi(close=df['close_price'])
    df['stoch'] = ta.momentum.stoch(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['tsi'] = ta.momentum.tsi(close=df['close_price'])
    df['uo'] = ta.momentum.ultimate_oscillator(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['wri'] = ta.momentum.williams_r(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    # volume
    df['accdist'] = ta.volume.acc_dist_index(
        high=df['high_price'], low=df['low_price'], close=df['close_price'], volume=df['volume'])
    df['cmf'] = ta.volume.chaikin_money_flow(
        high=df['high_price'], low=df['low_price'], close=df['close_price'], volume=df['volume'])
    df['emv'] = ta.volume.ease_of_movement(
        high=df['high_price'], low=df['low_price'], volume=df['volume'])
    df['fi'] = ta.volume.force_index(
        close=df['close_price'], volume=df['volume'])
    df['mfi'] = ta.volume.money_flow_index(
        high=df['high_price'], low=df['low_price'], close=df['close_price'], volume=df['volume'])
    df['nvi'] = ta.volume.negative_volume_index(
        close=df['close_price'], volume=df['volume'])
    df['obv'] = ta.volume.on_balance_volume(
        close=df['close_price'], volume=df['volume'])
    df['smaemv'] = ta.volume.sma_ease_of_movement(
        high=df['high_price'], low=df['low_price'], volume=df['volume'])
    df['vpt'] = ta.volume.volume_price_trend(
        close=df['close_price'], volume=df['volume'])
    df['vwap'] = ta.volume.volume_weighted_average_price(
        high=df['high_price'], low=df['low_price'], close=df['close_price'], volume=df['volume'])

    # volatility
    df['atr'] = ta.volatility.average_true_range(
        high=df['high_price'], low=df['low_price'], close=df['close_price'])
    df['ulcer'] = ta.volatility.ulcer_index(close=df['close_price'])

    df['bbh'] = ta.volatility.bollinger_hband(close=df['close_price'])
    df['bbl'] = ta.volatility.bollinger_lband(close=df['close_price'])
    df['bbhi'] = ta.volatility.bollinger_hband_indicator(
        close=df['close_price'])
    df['bbli'] = ta.volatility.bollinger_lband_indicator(
        close=df['close_price'])
    df['bbmavg'] = ta.volatility.bollinger_mavg(close=df['close_price'])
    df['bb_pb'] = ta.volatility.bollinger_pband(close=df['close_price'])
    df['bb_wb'] = ta.volatility.bollinger_wband(close=df['close_price'])

    df['dchb'] = ta.volatility.donchian_channel_hband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['dclb'] = ta.volatility.donchian_channel_lband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['dcmb'] = ta.volatility.donchian_channel_mband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['dcpb'] = ta.volatility.donchian_channel_pband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['dcwb'] = ta.volatility.donchian_channel_wband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    df['kchb'] = ta.volatility.keltner_channel_hband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kclb'] = ta.volatility.keltner_channel_lband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kchbi'] = ta.volatility.keltner_channel_hband_indicator(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kclbi'] = ta.volatility.keltner_channel_lband_indicator(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kcmb'] = ta.volatility.keltner_channel_mband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kcpb'] = ta.volatility.keltner_channel_pband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kcwb'] = ta.volatility.keltner_channel_wband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    # trend

    # En dos de estos tres hay algo que esta botando warning
    df['adx'] = ta.trend.adx(close=df['close_price'],
                             high=df['high_price'], low=df['low_price'])
    df['adx_neg'] = ta.trend.adx_neg(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['adx_pos'] = ta.trend.adx_pos(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    df['aroon_up'] = ta.trend.aroon_up(close=df['close_price'])
    df['aroon_down'] = ta.trend.aroon_down(close=df['close_price'])

    df['cci'] = ta.trend.cci(close=df['close_price'],
                             high=df['high_price'], low=df['low_price'])
    df['dpo'] = ta.trend.dpo(close=df['close_price'])
    df['kst'] = ta.trend.kst(close=df['close_price'])
    df['kst_sig'] = ta.trend.kst_sig(close=df['close_price'])
    df['ema'] = ta.trend.ema_indicator(close=df['close_price'])

    df['ichimoku_a'] = ta.trend.ichimoku_a(
        high=df['high_price'], low=df['low_price'])
    df['ichimoku_b'] = ta.trend.ichimoku_b(
        high=df['high_price'], low=df['low_price'])
    df['ichimoku_base_line'] = ta.trend.ichimoku_base_line(
        high=df['high_price'], low=df['low_price'])
    df['ichimoku_conversion_line'] = ta.trend.ichimoku_conversion_line(
        high=df['high_price'], low=df['low_price'])

    df['macd'] = ta.trend.macd(close=df['close_price'])
    df['macd_diff'] = ta.trend.macd_diff(close=df['close_price'])
    df['macd_signal'] = ta.trend.macd_signal(close=df['close_price'])

    df['mi'] = ta.trend.mass_index(high=df['high_price'], low=df['low_price'])
    df['sma'] = ta.trend.sma_indicator(close=df['close_price'])
    df['wma'] = ta.trend.wma_indicator(close=df['close_price'])
    df['stc'] = ta.trend.stc(close=df['close_price'])
    df['trix'] = ta.trend.trix(close=df['close_price'])

    df['vi_pos'] = ta.trend.vortex_indicator_pos(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['vi_neg'] = ta.trend.vortex_indicator_neg(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    return df

In [18]:
df_prueba = get_indicators_and_merge(base_url, EXCHANGE_ID, ASSET_ID, 1681603200, 1681948800)
df_prueba.sort_values(by='unix_time', ascending=True, inplace=True)
df_prueba.tail(15)

get asset status code: 200
symbol: BTCUSDT
interval: day


/Users/julianoliverosforero/Desktop/2023-1/Tesis/cryptocurrency-market-prediction-project/data-loader/env/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/julianoliverosforero/Desktop/2023-1/Tesis/cryptocurrency-market-prediction-project/data-loader/env/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


,taker_base_vol,open_price,taker_quote_vol,close_price,low_price,id,high_price,volume,unix_time,qav,date_time_utc,num_trades,ao,kama,ppo,pvo,roc,rsi,stochrsi,stoch,tsi,uo,wri,accdist,cmf,emv,fi,mfi,nvi,obv,smaemv,vpt,vwap,atr,ulcer,bbh,bbl,bbhi,bbli,bbmavg,bb_pb,bb_wb,dchb,dclb,dcmb,dcpb,dcwb,kchb,kclb,kchbi,kclbi,kcmb,kcpb,kcwb,adx,adx_neg,adx_pos,aroon_up,aroon_down,cci,dpo,kst,kst_sig,ema,ichimoku_a,ichimoku_b,ichimoku_base_line,ichimoku_conversion_line,macd,macd_diff,macd_signal,mi,sma,wma,stc,trix,vi_pos,vi_neg,corn_price,us_10_year_treasury_price,silver_price,wheat_price,us_5_year_treasury_price,natural_gas_price,soybean_price,us_2_year_treasury_price,cotton_price,oats_price,usbond_price,coffee_price,spy500_price,gold_price,sugar_price,dow_jones_price,cocoa_price,nasdaq_price,rice_price,russell_2000_price
0,13163.73591,30295.10,3.991156e+08,30304.65,30120.00,6c1e2c87-6d7c-4095-9f92-b9ef079155a0,30549.99,26431.99322,1681603200,8.013229e+08,2023-04-16T00:00:00+00:00,708233,4764.102000,25428.730774,7.493291,-27.702064,40.135879,61.402401,1.000000,97.303550,21.465394,71.818811,-2.696450,1.844478e+06,-0.044396,4.310882e+08,5.928906e+07,76.745661,1060.414225,2.850016e+06,9.258695e+08,4991.745297,25180.360490,2220.585401,6.892214,30968.414870,19593.254130,0.0,0.0,25280.8345,0.941648,44.995195,30550.0,21451.00,26404.00,0.970411,30.247869,26107.307500,24441.564500,1.0,0.0,25274.436000,3.519802,6.590624,17.617904,27.812957,48.988656,100.0,8.0,131.817580,-648.8845,221.060225,117.984861,27297.714302,25066.500,23729.00,23729.00,26404.00,1902.906738,578.998478,1323.908260,24.139339,26827.002500,28673.880444,NaN,0.714195,0.984020,0.792878,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,26880.17771,30304.66,7.977110e+08,29430.27,29240.65,309fa234-8fa3-49d2-8587-fef8177766af,30316.06,56441.81127,1681689600,1.675288e+09,2023-04-17T00:00:00+00:00,1137294,3184.990559,27214.112267,5.262353,-38.237974,22.049036,56.588947,0.743099,88.713511,12.172600,47.995433,-11.286489,2.209117e+06,0.130175,-8.506212e+08,4.345967e+07,65.248140,1154.940289,2.833007e+06,4.212767e+08,-462.793224,25169.006482,1940.425494,7.964896,32966.614062,20473.615938,0.0,0.0,26720.1150,0.716934,46.755031,30595.6,20042.72,29181.27,0.588028,9.731117,27931.630333,25444.281333,1.0,0.0,26687.955833,1.602505,9.320118,13.072638,31.311327,44.202500,96.0,36.0,73.168227,540.9550,191.127518,127.369194,28406.743264,27250.215,25521.36,25319.16,29181.27,1420.130749,271.794336,1148.336413,25.644359,28766.165000,29287.315556,99.998475,0.647582,1.002246,0.790113,676.50,114.359375,25.052999,889.75,109.101562,2.275,1517.00,102.972656,83.300003,342.00,129.75000,201.000000,4176.75,1994.199951,24.440001,34129.0,3003.0,13186.50,1733.0,1814.199951
1,32417.57068,29430.27,9.727696e+08,30380.01,29096.78,431338a7-ffc9-49b4-8b30-5cf6c19a5a0f,30485.00,62004.89434,1681776000,1.861001e+09,2023-04-18T00:00:00+00:00,1141562,4925.421471,25930.744305,7.793143,-30.607508,23.335789,61.534464,1.000000,98.131773,22.312905,66.955516,-1.868227,1.897104e+06,-0.049741,-1.218190e+09,5.148672e+07,69.678435,1060.414225,2.912020e+06,7.945952e+08,245.414607,25491.015234,2161.130730,5.848160,31643.484808,19664.540192,0.0,0.0,25654.0125,0.894525,46.694234,30550.0,21451.00,26404.00,0.979500,29.513080,26498.807500,24762.242500,1.0,0.0,25630.525000,3.234988,6.775378,17.928306,29.911432,46.748698,100.0,4.0,107.372489,-2496.9425,240.934072,140.769917,27771.913640,25066.500,23729.00,23729.00,26404.00,2007.831821,547.138849,1460.692972,24.238470,27306.007500,29181.434444,NaN,0.802585,0.947753,0.873520,677.50,114.484375,25.247999,881.25,109.164062,2.366,1519.25,102.949219,84.550003,347.50,130.03125,205.250000,4180.00,2007.400024,24.540001,34124.0,3054.0,13193.00,1734.0,1806.000000
2,41467.78872,30380.01,1.217556e+09,28797.10,28520.00,a8e8b0c1-a7ce-4beb-94f4-e336ec3f6c06,30413.53,86575.48656,1681862400,2.541284e+09,2023-04-19T00:00:00+00:00,1699891,3882.657206,26085.678314,5.094566,-23.821837,32.672

In [ ]:
df_prueba.tail(15)